# 导入相关库

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
import datetime
from tqdm import trange
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings
import time
from PIL import Image, ImageDraw, ImageFont
from sklearn.linear_model import  *
from matplotlib import pyplot as plt 
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.model_selection import KFold, RepeatedKFold
from xgboost import *
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.preprocessing import *
from sklearn.linear_model import  *
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sympy import *
from xgboost import plot_importance
warnings.filterwarnings('ignore')

C:\Users\Ksama\AppData\Local\conda\conda\envs\ml\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#查看结果的最大值，最小值，平均值
def show(result):
    print('max: ',str(max(result)))
    print('min: ',str(min(result)))
    print('mean: ',str((sum(result))/(len(result))))
    print()

In [3]:
#官方的loss评估
def logloss(y_true, y_pred,deta = 1.85, eps=1e-15):
    # Prepare numpy array data
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    assert (len(y_true) and len(y_true) == len(y_pred))
    # Clip y_pred between eps and 1-eps
    p = np.clip(y_pred, eps, 1-eps)
    loss = np.sum(- y_true * np.log(p) * deta - (1 - y_true) * np.log(1-p))
    return loss / len(y_true)

In [4]:
"""
卡方特征选取
"""
def chi_square_test(X,y,select_k=10):
    if select_k >= 1:
        sel_ = SelectKBest(k=select_k).fit(X,y)
        col = X.columns[sel_.get_support()]
    elif 0 < select_k < 1:
        sel_ = SelectPercentile(percentile=select_k*100).fit(X,y)
        col = X.columns[sel_.get_support()]   
    else:
        raise ValueError("select_k must be a positive number")  
    
    return col

In [5]:
'''
把结果保存成指定格式的文件
示例：save_result2csv(y_submit,submit,'./round1_diac2019_test.csv')
其中y_submit为模型输出结果，submit为读入数据时生成的一个变量，第三个参数为保存的文件路劲
'''
def save_result2csv(ys_submit,submit,csv_name):
    all_customers = pd.DataFrame(trian[['customer_id']]).drop_duplicates(['customer_id']).dropna()
    submits_df = submit[['customer_id']]
    submits_df['result'] = ys_submit
    all_customers = pd.merge(all_customers,submits_df,on=['customer_id'],how='left',copy=False)
    all_customers = all_customers.sort_values(['customer_id'])
    all_customers['customer_id'] = all_customers['customer_id'].astype('int64')
    all_customers['result'] = all_customers['result'].fillna(0)
    all_customers.to_csv(csv_name,index=False)

In [6]:
train = pd.read_csv('../Feature/train.csv',low_memory=False)
submit = pd.read_csv('../Feature/submit12.csv',low_memory=False)

In [7]:
y = train.pop('label')
feature = [x for x in train.columns if x not in ['customer_id']]
X = train[feature]

# 第一次训练

In [8]:
#第一次训练，利用卡方特征选取21个特征进行训练LGB和XGB
x_columns = chi_square_test(X, y, 21)
print(x_columns)
X_train, X_valid, y_train, y_valid = train_test_split(X[x_columns], y, test_size=0.25, random_state=42,stratify=y)
submit_df = submit[['customer_id']]
X_submit = submit[x_columns]

Index(['count', 'goods_price_mean', 'goods_price_min', 'goods_price_max',
       'goods_price_sum', 'is_member_actived', 'customer_gender',
       'order_month_count', 'goods_id_count', 'GDP', 'pay_deleta_time_last',
       'pay_deleta_time_first', 'pay_deleta_time_mean', 'goods_list_time_mean',
       'goods_list_time_last', 'goods_num', 'order_total_num',
       'order_total_payment', 'customer_province_l', 'order_pay_date_last',
       'long_time'],
      dtype='object')


In [9]:
#5折 训练lgb 这里在数据层面增强了正样本比例，但没有更换loss function
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1234)
y_pp_lgb = np.zeros(len(X_submit))
y_pp_lgb_stacking = np.zeros(len(y_train))
for train_index, test_index in kf.split(X_train):
    print ( ">>>", train_index )
    lightgbm = lgb.LGBMClassifier(learning_rate =0.01,
                                 n_estimators=300,
                                 max_depth=12,
                                 min_child_weight=1,
                                 eta = 0.01,
                                 alpha = 0.01,
                                 gamma=0.0,
                                 subsample=0.7,
                                 colsample_bytree=0.7,
                                 nthread=6,
                                 scale_pos_weight=1,
                                 seed=66)
    lightgbm_model = lightgbm.fit(X_train.iloc[train_index], y_train.iloc[train_index]) 

    y_pred = lightgbm_model.predict(X_train.iloc[test_index]) 
    y_predprob = lightgbm_model.predict_proba(X_train.iloc[test_index])[:, 1] 
    
    y_pp_lgb_stacking[test_index] = y_predprob
    
    print(metrics.confusion_matrix(y_train.iloc[test_index].values, y_pred))
    print("Accuracy : %.4g" % metrics.accuracy_score(y_train.iloc[test_index].values, y_pred))  
    auc = metrics.roc_auc_score(y_train.iloc[test_index], y_predprob)
    print("AUC Score (Train): %f" % auc) 
    loglossscore = logloss(y_train.iloc[test_index],y_predprob)
    print("logloss SCore: %f" %loglossscore)
    
    y_pp_lgb += lightgbm_model.predict_proba(X_submit)[:, 1] / n_splits 

>>> [      0       1       2 ... 1368473 1368475 1368478]
[[217501   1115]
 [ 52978   2102]]
Accuracy : 0.8024
AUC Score (Train): 0.790633
logloss SCore: 0.601484
>>> [      0       4       5 ... 1368476 1368477 1368478]
[[217420   1070]
 [ 53304   1902]]
Accuracy : 0.8013
AUC Score (Train): 0.792224
logloss SCore: 0.601541
>>> [      1       2       3 ... 1368476 1368477 1368478]
[[217143   1092]
 [ 53550   1911]]
Accuracy : 0.8004
AUC Score (Train): 0.792358
logloss SCore: 0.604049
>>> [      0       1       2 ... 1368476 1368477 1368478]
[[217034   1177]
 [ 53435   2050]]
Accuracy : 0.8005
AUC Score (Train): 0.791532
logloss SCore: 0.603770
>>> [      0       1       2 ... 1368475 1368476 1368477]
[[217195   1230]
 [ 53195   2075]]
Accuracy : 0.8011
AUC Score (Train): 0.790400
logloss SCore: 0.603393


# 第二次训练

In [10]:
p_x = lightgbm_model.predict_proba(X[x_columns])[:,1]
p_submit = y_pp_lgb

In [11]:
X0 = pd.DataFrame()
X0 = X
X0['p']=p_x
X_train0, X_valid0, y_train0, y_valid0 = train_test_split(X0, y, test_size=0.25, random_state=42,stratify=y)
submit_df = submit[['customer_id']]
X_submit00 = submit[feature]
X_submit01 = X_submit00[x_columns]
X_submit01['p']=p_submit

In [12]:
#8折 训练lgb 这里在数据层面增强了正样本比例，但没有更换loss function
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1234)
y_pp_lgb0 = np.zeros(len(X_submit01))
y_pp_lgb_stacking0 = np.zeros(len(y_train0))
for train_index, test_index in kf.split(X_train0):
    print ( ">>>", train_index )
    lightgbm0 = lgb.LGBMClassifier(learning_rate =0.01,
                                 n_estimators=200,
                                 max_depth=12,
                                 min_child_weight=1,
                                 eta = 0.01,
                                 alpha = 0.01,
                                 gamma=0.0,
                                 subsample=0.7,
                                 colsample_bytree=0.7,
                                 nthread=6,
                                 scale_pos_weight=1,
                                 seed=66)
    lightgbm_model0 = lightgbm0.fit(X_train0.iloc[train_index], y_train0.iloc[train_index]) 

    y_pred = lightgbm_model0.predict(X_train0.iloc[test_index]) 
    y_predprob = lightgbm_model0.predict_proba(X_train0.iloc[test_index])[:, 1] 
    
    y_pp_lgb_stacking0[test_index] = y_predprob
    
    print(metrics.confusion_matrix(y_train0.iloc[test_index].values, y_pred))
    print("Accuracy : %.4g" % metrics.accuracy_score(y_train0.iloc[test_index].values, y_pred))  
    auc = metrics.roc_auc_score(y_train0.iloc[test_index], y_predprob)
    print("AUC Score (Train): %f" % auc) 
    loglossscore = logloss(y_train0.iloc[test_index],y_predprob)
    print("logloss SCore: %f" %loglossscore)
    
    y_pp_lgb0 += lightgbm_model0.predict_proba(X_submit01)[:, 1] / n_splits 

>>> [      0       1       2 ... 1368473 1368475 1368478]
[[216068   2548]
 [ 50840   4240]]
Accuracy : 0.8049
AUC Score (Train): 0.795188
logloss SCore: 0.599545
>>> [      0       4       5 ... 1368476 1368477 1368478]
[[215999   2491]
 [ 51006   4200]]
Accuracy : 0.8045
AUC Score (Train): 0.796046
logloss SCore: 0.599296
>>> [      1       2       3 ... 1368476 1368477 1368478]
[[215738   2497]
 [ 51209   4252]]
Accuracy : 0.8038
AUC Score (Train): 0.796685
logloss SCore: 0.602078
>>> [      0       1       2 ... 1368476 1368477 1368478]
[[215647   2564]
 [ 51313   4172]]
Accuracy : 0.8032
AUC Score (Train): 0.796157
logloss SCore: 0.601606
>>> [      0       1       2 ... 1368475 1368476 1368477]
[[215714   2711]
 [ 51063   4207]]
Accuracy : 0.8035
AUC Score (Train): 0.794647
logloss SCore: 0.601761


# 保存预测结果

In [35]:
save_result2csv(y_pp_lgb0,submit,'../Res/res_new12.csv')

# 数据后处理

In [36]:
#把一年内购买次数超过10次的人，购买概率增加0.1
res = pd.read_csv('../Res/res_new12.csv')
id0=submit["customer_id"][submit["count"]>10]

id1=np.array(id0)
for i in trange(len(id1)):
    res["result"][res["customer_id"]==id1[i]] +=0.3


100%|██████████████████████████████████████████████████████████████████████████████| 2635/2635 [01:58<00:00, 22.17it/s]


In [37]:
res.to_csv('../Res/res_new12_final.csv',index=False)